In [ ]:

!pip install ultralytics opencv-python matplotlib --quiet

In [ ]:
from ultralytics import YOLO
import cv2
import os
import matplotlib.pyplot as plt

In [ ]:

DATASET_PATH = "/Users/kesavp/shell\ intern/idd20kII"

In [ ]:
yaml_content = """
path: /content/dataset
train: train/images
val: val/images

names:
  0: car
  1: truck
  2: bus
  3: motorcycle
  4: bicycle
  5: pedestrian
  6: animal
"""
with open("dataset.yaml", "w") as f:
    f.write(yaml_content)

print("✅ dataset.yaml created successfully!")

In [ ]:
model = YOLO("yolov8n.pt")  

model.train(
    data="dataset.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name="mini_tesla_model"
)

In [ ]:
metrics = model.val()
print("Validation Results:", metrics)

In [ ]:
TEST_IMAGE = "/content/dataset/val/images/sample.jpg"  

results = model.predict(source=TEST_IMAGE, show=True)

In [ ]:
img = cv2.imread(TEST_IMAGE)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Detected Objects")
plt.show()

In [ ]:
model.export(format="onnx")  # export to ONNX format for deployment

In [ ]:
# Run inference on a folder (all test images)
TEST_DIR = "/content/dataset/val/images"  # change as needed

results = model.predict(source=TEST_DIR, save=True, conf=0.5)
print("✅ Inference completed. Check the 'runs/detect' folder for output images.")

In [ ]:
# Print key evaluation metrics
print("📊 Model Performance Metrics:")
print(f"Precision: {metrics.box.map:.3f}")
print(f"mAP@50: {metrics.box.map50:.3f}")
print(f"mAP@50-95: {metrics.box.map75:.3f}")

In [ ]:
import glob
import random

# Pick a random result image from YOLO’s output folder
output_dir = "runs/detect/predict"
sample_img = random.choice(glob.glob(os.path.join(output_dir, "*.jpg")))

# Display the detection
img = cv2.imread(sample_img)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Mini Tesla Detection Result")
plt.show()

In [ ]:
# Save final trained weights for reuse or deployment
model_path = "best_mini_tesla.pt"
model.save(model_path)
print(f"✅ Model saved successfully as {model_path}")

In [ ]:

 cap = cv2.VideoCapture(0)  # webcam
 while True:
     ret, frame = cap.read()
     if not ret:
         break
     results = model(frame)
     annotated_frame = results[0].plot()
     cv2.imshow("Mini Tesla Live Detection", annotated_frame)
     if cv2.waitKey(1) & 0xFF == ord('q'):
         break
cap.release()
cv2.destroyAllWindows()

In [ ]:
print("""
🎯 MINI TESLA OBJECT DETECTION COMPLETED SUCCESSFULLY!
-----------------------------------------------------
✅ Dataset Used: Indian Driving Dataset (IDD)
✅ Model: YOLOv8 Nano (custom-trained)
✅ Detected Objects: Car, Truck, Bus, Motorcycle, Bicycle, Pedestrian, Animal
✅ Goal: Real-time detection system for Indian road conditions
""")